This example shows how to access data from the RACE database in Julia, using the [`LibPQ`](https://github.com/invenia/LibPQ.jl) library.

Note that you have to be connected to the VPN.

In [31]:
using LibPQ
using Tables
using PyPlot
using PyCall
using Dates
const plt = PyPlot
mdates = pyimport("matplotlib.dates")

PyObject <module 'matplotlib.dates' from '/home/ctroupin/.julia/conda/3/lib/python3.9/site-packages/matplotlib/dates.py'>

### Configuration

In [32]:
plt.rc("font", family="Times New Roman", size=16)
figdir = "../../figures/"
isdir(figdir) ? @debug("ok") : mkpath(figdir);

## Establish the connection
The user name and host are stored in a text file (`.bashrc` for example) that has to be sourced before running the notebook.       
You will be prompted to enter your password. 

In [54]:
conn = LibPQ.Connection("dbname=race_db_v2 user=$(ENV["RACEuser"]) host=$(ENV["RACEhost"])");

Enter password for PostgreSQL user master:: ········


## Create query
We set the initial and final date of the period of interest, as well as the variable name (`paramname`) and the station name. 
The list of available parameters and stations can be found at http://www.gitan.ulg.ac.be/Intranet/Intranet_BDRACE.html.    

For the stations, the name has to be one of these:
* the weather mast → `INSJD`
* Roof of STARESO → `INST2`
* STARESO harbour → `INST1` and `NEW03`
* Optode mooring in front of the station → `OPT01`

In [53]:
dateinit = DateTime(2022, 1, 1)
dateend = DateTime(2022, 4, 20)
paramname = "airtemp"
stationname = "INSJD"

"INSJD"

In [55]:
function get_query(paramname::String, stationname::String="", dateinit::DateTime=DateTime(1900, 1, 1), 
        dateend::DateTime=DateTime(2100, 1, 1))
    
    stationlist = ["INSJD", "INST2", "INST1", "NEW03", "OPT01"]
    
    if stationname ∈ stationlist

        thequery = """SELECT
          data.datavalue,
          data.daytime
        FROM
          public.data,
          public.timeseries
        WHERE
          data.dasa_codeds = timeseries.codeds AND
          timeseries.tssta_codesta = '$(stationname)' AND
          data.daytime >= '$(Dates.format(dateinit, "yyyy-mm-dd HH:MM:SS"))' AND
          data.daytime <= '$(Dates.format(dateend, "yyyy-mm-dd HH:MM:SS"))' AND
          data.dataparam_codeparam = '$(paramname)'
        GROUP BY
          data.dataparam_codeparam,
          data.datavalue,
          data.daytime
        ORDER BY
          data.daytime;
          """
        thequery = replace(thequery, "\n" => " ");
        
        return thequery
    elseif length(stationname) == 0
        @error("You did not specity any station name")
    else
        @error("The station you specified doesn't exist")
    end
end

get_query (generic function with 6 methods)

In [42]:
temperature_query = get_query("airtemp", "INSJD", dateinit, dateend);

In [43]:
result = execute(conn, temperature_query)
data = columntable(result);

Close connection

In [44]:
close(conn)

## Create plot

In [48]:
function plot_timeseries(data, paramname, dateinit, dateend)
    fig = plt.figure(figsize=(12, 6))
    ax = plt.subplot(111)
    ax.plot(data[:daytime], data[:datavalue], "k", lw=.5)
    ax.set_ylabel("Air temperature\n(°C)", rotation=0, ha="right")
    ax.plot(data[:daytime][argmax(data[:datavalue])], maximum(data[:datavalue]), "ro", ms=7)
    ax.plot(data[:daytime][argmin(data[:datavalue])], minimum(data[:datavalue]), "bo", ms=7)
    #ax.text(0.01, 0.05, minvaluetext, transform=ax.transAxes)
    #ax.text(0.01, 0.10, maxvaluetext, transform=ax.transAxes)
    ax.set_xlim(dateinit, dateend)
    ax.xaxis.set_minor_locator(mdates.DayLocator())
    ax.spines.right.set_visible(false)
    ax.spines.top.set_visible(false)
    # ax.set_title("Air temperature in 2022")
    fig.autofmt_xdate()
    plt.savefig(joinpath(figdir, "$(paramname)_$(Dates.format(dateinit, "yyyymmdd"))_$(Dates.format(dateend, "yyyymmdd")).jpg"), dpi=300, bbox_inches="tight")
    #plt.show()
    plt.close()
end

plot_timeseries (generic function with 1 method)

## Loop on years

In [73]:
for yy = 1988:2022
    @info("Working on year $(yy)");
    dateinit = DateTime(yy, 1, 1)
    dateend = DateTime(yy, 12, 31)
    temperature_query = get_query("airtemp", "INSJD", dateinit, dateend);
    result = execute(conn, temperature_query)
    @info(result.not_null);
    
    data = columntable(result);
    if length(data[:daytime]) == 0
        @info("No data, won't plot anything")
    else
        plot_timeseries(data, paramname, dateinit, dateend)
    end
end

┌ Info: Working on year 1988
└ @ Main In[73]:2
┌ Info: Bool[0, 0]
└ @ Main In[73]:7
┌ Info: No data, won't plot anything
└ @ Main In[73]:11
┌ Info: Working on year 1989
└ @ Main In[73]:2
┌ Info: Bool[0, 0]
└ @ Main In[73]:7
┌ Info: No data, won't plot anything
└ @ Main In[73]:11
┌ Info: Working on year 1990
└ @ Main In[73]:2
┌ Info: Bool[0, 0]
└ @ Main In[73]:7
┌ Info: No data, won't plot anything
└ @ Main In[73]:11
┌ Info: Working on year 1991
└ @ Main In[73]:2
┌ Info: Bool[0, 0]
└ @ Main In[73]:7
┌ Info: No data, won't plot anything
└ @ Main In[73]:11
┌ Info: Working on year 1992
└ @ Main In[73]:2
┌ Info: Bool[0, 0]
└ @ Main In[73]:7
┌ Info: No data, won't plot anything
└ @ Main In[73]:11
┌ Info: Working on year 1993
└ @ Main In[73]:2
┌ Info: Bool[0, 0]
└ @ Main In[73]:7
┌ Info: No data, won't plot anything
└ @ Main In[73]:11
┌ Info: Working on year 1994
└ @ Main In[73]:2
┌ Info: Bool[0, 0]
└ @ Main In[73]:7
┌ Info: No data, won't plot anything
└ @ Main In[73]:11
┌ Info: Working on y

LoadError: PyError ($(Expr(:escape, :(ccall(#= /home/ctroupin/.julia/packages/PyCall/ygXW2/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'TypeError'>
TypeError("float() argument must be a string or a number, not 'PyCall.jlwrap'")
  File "/home/ctroupin/.julia/conda/3/lib/python3.9/site-packages/matplotlib/axes/_axes.py", line 1637, in plot
    self.add_line(line)
  File "/home/ctroupin/.julia/conda/3/lib/python3.9/site-packages/matplotlib/axes/_base.py", line 2288, in add_line
    self._update_line_limits(line)
  File "/home/ctroupin/.julia/conda/3/lib/python3.9/site-packages/matplotlib/axes/_base.py", line 2311, in _update_line_limits
    path = line.get_path()
  File "/home/ctroupin/.julia/conda/3/lib/python3.9/site-packages/matplotlib/lines.py", line 999, in get_path
    self.recache()
  File "/home/ctroupin/.julia/conda/3/lib/python3.9/site-packages/matplotlib/lines.py", line 657, in recache
    y = _to_unmasked_float_array

In [60]:
yy = 2010
dateinit = DateTime(yy, 1, 1)
dateend = DateTime(yy, 4, 20)
temperature_query = get_query("airtemp", "INSJD", dateinit, dateend);
result = execute(conn, temperature_query)

PostgreSQL result

In [67]:
typeof(result)

LibPQ.Result{false}